# Lab 2

In [1]:
import numpy as np
import scipy.signal as sig

import plotly.graph_objects as go

In [2]:
def plot(x,y,name_):
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t, y=y))
    fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')
    fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')
    fig.update_layout(legend_orientation="h",
                    title=name_,
                    legend=dict(x=.5, xanchor="center")
                    #margin=dict(l=0, r=0, t=0, b=0)
                    )

    #fig.update_yaxes(range=[-500,200])
    #fig.update_xaxes(range=[-0.5, 10])
    return fig.show()

## Transfer function representation

In [3]:
num = np.array([1,-1,1,-1])
denom_un = np.array([1,0,1,4])
denom_st = np.array([1,5,1,4])

## Loop 


In [4]:
def add_loop(numerator, denominator, k):
     return (k*num, denom- k*num)

def sub_loop(numerator, denominator, k):
    return (k*num, denom + k*num)

In [5]:
print(f"Physical implementability:  {num.shape[0] <= denom_st.shape[0]}")

Physical implementability:  True


## ZPK

### Unstable system

In [6]:
z,p,k = sig.tf2zpk(num, denom_un)

In [29]:
def print_zpk(z, p, k):
    print(f"Zeros: {np.round(z, 2)}",f"Poles: {np.round(p, 2)}",f"Poles: {np.round(p, 2)}",f"Koeff: {k}",sep="\n")

    print(f"System stability: {~np.any(p.real > 0)}")

In [30]:
print_zpk(z, p, k)

Zeros: [1.+0.j 0.+1.j 0.-1.j]
Poles: [-4.96+0.j  -0.02+0.9j -0.02-0.9j]
Poles: [-4.96+0.j  -0.02+0.9j -0.02-0.9j]
Koeff: 1.0
System stability: True


In [9]:
t,y = sig.step((num, denom_un))

plot(t, y,name_="Step response")


In [10]:
t,y = sig.impulse((num,denom_un))

plot(t, y,"Impulse response")

### Stable system

In [11]:
z,p,k = sig.tf2zpk(num, denom_st)

In [12]:
t,y = sig.step((num, denom_st))

plot(t, y,"Step response")


In [13]:
t,y = sig.impulse((num, denom_st))

plot(t, y,"Impulse response")

In [14]:
print_zpk(z, p, k)

Zeros: [1.+0.j 0.+1.j 0.-1.j]
Poles: [-4.96+0.j  -0.02+0.9j -0.02-0.9j]
Koeff: 1.0
System stability: True


## Space-state representation

$q$ - state variables

$$
\begin{cases}
    \dfrac{dq}{dt} &= \textbf{A}q + \textbf{B}x \\
    y &= \textbf{C}q + \textbf{D}x
\end{cases}
$$

In [15]:
A,B,C,D = sig.zpk2ss(z, p, k)

In [27]:
def observable(A,B,C,D):    
    T = np.empty_like(A)
    for i in range(A.shape[0]):
        T[i, :] = (np.dot(np.linalg.matrix_power(A, i).T, C.T)).reshape(A.shape[0])    
    
    An = T.dot(A).dot(np.linalg.inv(T))
    Bn = np.dot(T,B)
    Cn = np.dot(C,np.linalg.inv(T))    
    return An, Bn, Cn, D

def controlable(A,B, C, D):
    T = np.empty_like(A)
    for i in range(A.shape[0]):
        T[:, i] = (np.linalg.matrix_power(A, i).dot(B)).reshape(A.shape[0])    
        
    An = np.linalg.inv(T).dot(A).dot(T)
    Bn = np.dot(np.linalg.inv(T),B)
    Cn = np.dot(C,T)
    return An, Bn, Cn, D

In [25]:
print("Observable canonical form")
print(np.round(observable(A, B, C, D)[0], 3))

Observable canonical form
[[-0.  1. -0.]
 [-0. -0.  1.]
 [-4. -1. -5.]]


In [32]:
print("Controlable canonical form")
print(np.round(controlable(A, B, C, D)[0], 3))

Controlable canonical form
[[ 0.  0. -4.]
 [ 1.  0. -1.]
 [ 0.  1. -5.]]


System obsevable if

$ \mathrm{rank} 
\begin{pmatrix} 
    C \\ 
    CA \\
    CA^2 \\
    \dots \\
    CA^{n-1}
\end{pmatrix} = n $

System controlable if

$ \mathrm{rank} 
\begin{pmatrix} 
    B \;
    AB \;
    A^2B \;
    \dots \; 
    A^{n-1}B
\end{pmatrix} = n$

In [19]:
def validate_observable(A, C):
    T = np.empty_like(A)
    for i in range(A.shape[0]):
        T[:, i] = (C @ np.linalg.matrix_power(A, i)).reshape(A.shape[0])
    return np.linalg.matrix_rank(T) == A.shape[0]

def validate_controlable(A, B):
    T = np.empty_like(A)
    for i in range(A.shape[1]):
        T[i, :] = (np.linalg.matrix_power(A, i) @ B).reshape(A.shape[0])
    return np.linalg.matrix_rank(T) == A.shape[0]

In [20]:
print(f"System controlability: {validate_controlable(A, B)}")
print(f"System observability: {validate_observable(A, C)}")

System controlability: True
System observability: True


In [21]:
num2, denom2 = sig.ss2tf(A,B,C,D)

In [22]:
t, y1 = sig.step((num, denom_st))
t, y2 = sig.step((num2, denom2))

plot(t,y1 - y2,"Difference of Step response")

In [23]:
t,y1 = sig.impulse((num, denom_st))
t, y2 = sig.impulse((num2, denom2))

plot(t, y1 - y2,"Difference of Impulse response")